In [0]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

from operator import add
from functools import reduce

from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, Lasso, LinearRegression

from sklearn.model_selection import train_test_split, GridSearchCV

# Régression

* modéliser des relations statistiques entre deux variables

* estimer l'impact de la variation d'une variable explicative sur une variable à  expliquer

* modèle : $y = f(x) + \epsilon$

# Partie I : Données simulées

##I.1. Régression linéaire

* $f(x) = \langle x, \beta \rangle$

* estimer le vecteur de paramètre $\beta$

a) Générer des données suivant le modèle : 
$$ y_i = \beta_0 + \beta_1 x_i + \epsilon_i $$

In [0]:
def linear_data(n_samples, variance):
    # generate 2D data
    X = np.random.random((n_samples, 1))

    # generate two random coefficients that will be our model generator
    # we want to find them back using different algorithms
    W = np.random.random((2, 1))

    # construct targets with our data and weights
    y = W[0] + W[1] * X

    # a more elegant way could be to add a vector of ones to ease calculus
    # X = np.concatenate((np.ones((n_samples, 1)), X), axis=1)
    # and then to construct target
    # y = W @ X
    # ---
    # but that shape is a bit tricky to plot later...

    # add some noise to our data
    y += np.random.normal(0, variance, (n_samples, 1))

    return X, y, W

b) Afficher les données et la droite qui a permis de générer les données.

c) Calculer la droite de régression par la méthode des moindres carrés et l'afficher.

In [0]:
def ols_regression(X, y, fit_intercept=True, degree=1):
    # universal ols regression, suitable for any polynomial data
    X = np.concatenate(([X ** i for i in range(1, degree + 1)]), axis=1)

    # add a ones vector at first position
    if fit_intercept:
        X = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

    return np.linalg.inv(X.T @ X) @ X.T @ y


def get_regression_predictions(X, weights):
    # helper function to return predictions from data and weights
    # suitable for any polynomial data
    weights = weights.squeeze()
    return reduce(add, [weights[i] * (X ** i) for i in range(len(weights))])

In [0]:
 # generate some noisy linear data
n_samples, variance = 100, 0.01
X, Y, W = linear_data(n_samples, variance)

# we'll plot data between min and max
x_range = np.linspace(X.min(), X.max(), n_samples).reshape(-1, 1)

# calculate the regression line with ols regression
ols_weights = ols_regression(X, Y, fit_intercept=True, degree=1)

# get the line with found weights according to x_range
ols_line = get_regression_predictions(x_range, ols_weights)

# get the line with true weights according to x_range
true_line = get_regression_predictions(x_range, W)

# ploting power !
fig = go.Figure(
    data=[
        # markers
        go.Scatter(
            x=X.squeeze(),
            y=Y.squeeze(),
            mode='markers',
            name=f'linear data [{n_samples}, {variance}]'
        ),
        # true line
        go.Scatter(
            x=x_range.squeeze(),
            y=true_line.squeeze(),
            mode='lines',
            name=f"true line {W.squeeze()}"
        ),
        # ols found line
        go.Scatter(
            x=x_range.squeeze(),
            y=ols_line.squeeze(),
            mode='lines',
            name=f"ols found line {ols_weights.squeeze()}"
        )
    ],
    layout={
        'legend': {
            'orientation': 'h'
        }
    }
)
fig.show()

##I.2. Régression non linéaire


Nous allons étudier le cas de données périodiques. Générer des données suivant le modèle : 
$$ y_n = \sin(\frac{n}{10}) +(\frac{n}{50})^2 + \epsilon $$

In [0]:
def periodic_data(n_samples, variance, periodic_func, start=0, stop=10):
    # generate a 2D data line with `n_samples` points between `start` and `stop`
    X = np.linspace(start, stop, n_samples)

    # construct targets with our data and periodic function
    Y = periodic_func(X)

    # add some noise to our data
    Y += np.random.normal(0, variance, n_samples) * stop

    X = X.reshape((n_samples, 1))

    return X, Y

### I.2.1. Régression polynomiale

* $y = \beta_0 + \beta_1 x + \beta_2 x^2 +\beta_3 x^3$

* La solution d'une régression linéaire : $\hat{\beta} = (X^\top X)^{-1} X^\top Y$

* Utiliser $X = [1 \ x \ x^2 \ x^3]$


In [0]:
# that param can be interpreted as a regularization for the periodic function
# if it matches the max_range of the periodic function, then we'll consider
# only a `nearly linear` portion of the function.
# thus the polynomial regression will be very efficient
# ---
# in case that parameter is 1, then the polynomial regression will not work well
PARAM_TO_CHANGE = 1

# generate some noisy periodic data
n_samples, variance, start, stop = 200, 0.01, 0, 10
W = lambda var: np.sin(var/PARAM_TO_CHANGE) + (var/50) ** 2
X, Y = periodic_data(n_samples, variance, W, start, stop)

In [0]:
# calculate the regression line with ols regression
ols_weights = ols_regression(X, Y, fit_intercept=True, degree=3)

# we'll plot data between min and max
x_range = np.linspace(X.min(), X.max(), n_samples).reshape(-1, 1)

# get the line with found weights according to x_range
ols_line = get_regression_predictions(x_range, ols_weights)

# ploting power !
fig = go.Figure(
    data=[
        # markers
        go.Scatter(
            x=X.squeeze(),
            y=Y.squeeze(),
            mode='markers',
            name=f'linear data [{n_samples}, {variance}]'
        ),
        # true function
        go.Scatter(
            x=x_range.squeeze(),
            y=W(x_range).squeeze(),
            mode='lines',
            name=f"Generative function sin(x/{PARAM_TO_CHANGE}) + (x/50) + e"
        ),
        # ols found line
        go.Scatter(
            x=x_range.squeeze(),
            y=ols_line.squeeze(),
            mode='lines',
            name=f"ols found line {ols_weights.squeeze()}"
        )
    ],
    layout={
        'legend': {
            'orientation': 'h'
        }
    }
)
fig.show()

###I.2.2 Regression à noyau

a) Dans le cas du modèle linéaire, appliquer la régression à noyau avec un noyau linéaire. Utiliser la fonction KernelRidge de Scikit-learn avec un noyau linéaire. Représenter sur un graphique les données et la solution obtenue.

In [0]:
# we define a function to concatenate a ones vector because
# KernelRidge() doesn't have a fit_intercept parameter
def vector_prefixed_by_ones_vector(X):
    return np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)

In [0]:
 # generate some noisy linear data
n_samples, variance = 100, 0.01
X, Y, W = linear_data(n_samples, variance)

# use the Ridge classifier
ridge = Ridge()
ridge.fit(X, Y)

# use the KernelRidge classifier
kernelRidge = KernelRidge()
X_with_ones = vector_prefixed_by_ones_vector(X)
kernelRidge.fit(X_with_ones, Y)

# we'll plot data between min and max
x_range = np.linspace(X.min(), X.max(), n_samples).reshape(-1, 1)
x_range_with_ones = vector_prefixed_by_ones_vector(x_range)

# get the line with true weights according to x_range
true_line = get_regression_predictions(x_range, W)

# ploting power !
fig = go.Figure(
    data=[
        # markers
        go.Scatter(
            x=X.squeeze(),
            y=Y.squeeze(),
            mode='markers',
            name=f'linear data [{n_samples}, {variance}]'
        ),
        # true line
        go.Scatter(
            x=x_range.squeeze(),
            y=true_line.squeeze(),
            mode='lines',
            name=f"true line {W.squeeze()}"
        ),
        # ridge found line
        go.Scatter(
            x=x_range.squeeze(),
            y=ridge.predict(x_range).squeeze(),
            mode='lines',
            name=f"Ridge predictions"
        ),
        # kernel ridge found line
        go.Scatter(
            x=x_range.squeeze(),
            y=kernelRidge.predict(x_range_with_ones).squeeze(),
            mode='lines',
            name=f"Kernel Ridge predictions"
        )
    ],
    layout={
        'legend': {
            'orientation': 'h'
        }
    }
)
fig.show()

b) Dans le cas du modèle non-linéaire, appliquer la régression à noyau avec un noyau rbf. Représenter sur un graphique les données et la solution obtenue. 

In [0]:
# that param can be interpreted as a regularization for the periodic function
# if it matches the max_range of the periodic function, then we'll consider
# only a `nearly linear` portion of the function.
# thus the polynomial regression will be very efficient
# ---
# in case that parameter is 1, then the polynomial regression will not work well
PARAM_TO_CHANGE = 1

# generate some noisy periodic data
n_samples, variance, start, stop = 200, 0.01, 0, 10
W = lambda var: np.sin(var/PARAM_TO_CHANGE) + (var/50) ** 2
X, Y = periodic_data(n_samples, variance, W, start, stop)

In [0]:
# use the KernelRidge classifier with a rbf kernel
rbf_kernel_ridge = KernelRidge(kernel='rbf')
X_with_ones = vector_prefixed_by_ones_vector(X)
rbf_kernel_ridge.fit(X_with_ones, Y)

# we'll plot data between min and max
x_range = np.linspace(X.min(), X.max(), n_samples).reshape(-1, 1)
x_range_with_ones = vector_prefixed_by_ones_vector(x_range)

# ploting power !
fig = go.Figure(
    data=[
        # markers
        go.Scatter(
            x=X.squeeze(),
            y=Y.squeeze(),
            mode='markers',
            name=f'linear data [{n_samples}, {variance}]'
        ),
        # true function
        go.Scatter(
            x=x_range.squeeze(),
            y=W(x_range).squeeze(),
            mode='lines',
            name=f"Generative function sin(x/{PARAM_TO_CHANGE}) + (x/50) + e"
        ),
        # kernel ridge found line
        go.Scatter(
            x=x_range.squeeze(),
            y=rbf_kernel_ridge.predict(x_range_with_ones).squeeze(),
            mode='lines',
            name=f"Kernel Ridge predictions"
        )
    ],
    layout={
        'legend': {
            'orientation': 'h'
        }
    }
)
fig.show()

# **Conclusions partie I**

* dans le cas de données linéairement séparables, des fonctions polynomiales approchent très bien les poids de la fonction générative

* dans le cas de données périodiques - donc non linéairement séparables - des fonctions polynomiales peuvent bien prédirent des intervales réduits de cette fonction - en fait les portions linéaires

* dans le cas d'une généralisation à l'ensemble de la fonction périodique, les régressions polynomiales ne sont pas efficaces

* il convient alors d'utiliser d'autres modèles, comme par exemple le KernelRidge en utilisant le `kernel trick`, par exemple ici un noyau rbf

# Partie II : Données réelles

Soit les données suivantes : https://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset#

In [0]:
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
! unzip 'Bike-Sharing-Dataset.zip'

df = pd.read_csv('day.csv')
target = 'cnt'
Y = df[target]
X = df.drop([target, 'instant', 'dteday', 'yr', 'casual', 'registered'], axis=1)

--2019-11-01 14:17:34--  https://archive.ics.uci.edu/ml/machine-learning-databases/00275/Bike-Sharing-Dataset.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 279992 (273K) [application/x-httpd-php]
Saving to: ‘Bike-Sharing-Dataset.zip.3’

Bike-Sharing-Datase 100%[===================>] 273.43K  1.74MB/s    in 0.2s    

2019-11-01 14:17:34 (1.74 MB/s) - ‘Bike-Sharing-Dataset.zip.3’ saved [279992/279992]

Archive:  Bike-Sharing-Dataset.zip
replace Readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: Readme.txt              
replace day.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: day.csv                 
replace hour.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: hour.csv                


La régression ridge et lasso sont des extensions de la régression linéaire par moindres carrés permettant d'éviter le risque de sur-apprentissage. L'idée est d'ajouter une pénalisation au problème de régression par moindres carrés:
$$ \arg\min_{w \in \mathbb{R}^d, b \in \mathbb{R}} \sum_{i=1}^n \big(y_i - \langle w, x_i \rangle-b\big)^2 + \lambda \Omega(w) $$


$$ \text{avec} \qquad \lambda \in \mathbb{R} \qquad \text{le paramètre de régularisation où} $$

$$ \Omega(w) = \|w\|_2^2 \qquad \text{pour la régression Ridge} $$
$$ \Omega(w) = \|w\|_1 \qquad \text{pour la régression Lasso} $$

a) Utilisant la bibliothèque Scikit-learn, appliquez la régression Ridge et la régression Lasso sur le jeu de données "Bike Sharing". Affichez et comparez les deux solutions obtenues.

b) Calculez les erreurs de prédiction sur les données d'apprentissage obtenues avec les régressions Ridge et Lasso. Comparez les résultats avec ceux obtenus par la régression par moindre carrés.

In [0]:
# split into train/test
ratio, seed = 0.2, 42
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,
                                                    test_size=ratio,
                                                    random_state=seed)
alpha = 1

# initialize classifiers
classifiers = {
    f'ridge_alpha={alpha}': Ridge(alpha=alpha),
    f'lasso_alpha={alpha}': Lasso(alpha=alpha),
    'linear_reg': LinearRegression()
}

results = []

# train classifiers and predict both train/test data sets
for clf_name, clf in classifiers.items():
    clf.fit(X_train, Y_train)
    results.append({
        'name': clf_name,
        'train_score': clf.score(X_train, Y_train),
        'test_score':  clf.score(X_test, Y_test)
    })

results = pd.DataFrame(results)

In [0]:
# ploting power !
fig = go.Figure(
    data=[
        # train scores
        go.Bar(
            x=results['name'],
            y=results['train_score'],
            name='Train score',
            text=round(results['train_score'], 5),
            textposition='auto'
        ),
        # test scores
        go.Bar(
            x=results['name'],
            y=results['test_score'],
            name='Test score',
            text=round(results['test_score'], 5),
            textposition='auto'
        ),
    ],
    layout={
        'legend': {
            'orientation': 'h'
        }
    }
)
fig.show()

c)  Le choix du paramètre $\lambda$ est primordial pour avoir des résultats de prédiction optimaux. Une façon de procéder pour trouver une bonne valeur $\lambda$ est d'utiliser la méthode de cross-validation sur une grille de valeurs. Déterminer par cross-validation les valeurs de $\lambda$ permettant d'avoir les meilleurs taux de prédiction. 



In [0]:
# adapted code from
# http://www.davidsbatista.net/blog/2018/02/23/model_optimization/

class EstimatorSelectionHelper:
    def __init__(self, models, params):
        if not set(models.keys()).issubset(set(params.keys())):
            missing_params = list(set(models.keys()) - set(params.keys()))
            raise ValueError(
                f'Some estimators are missing parameters: {missing_params}')
        self.models = models
        self.params = params
        self.keys = models.keys()
        self.grid_searches = {}

    def fit(self, X, y, cv=3, n_jobs=3, verbose=1, scoring=None, refit=False):
        for key in self.keys:
            print(f'Running GridSearchCV for {key}')
            model = self.models[key]
            params = self.params[key]
            gs = GridSearchCV(model, params, cv=cv, n_jobs=n_jobs,
                              verbose=verbose, scoring=scoring, refit=refit,
                              return_train_score=True)
            gs.fit(X, y)
            self.grid_searches[key] = gs

    def score_summary(self, sort_by='mean_score',
                      num_rows_per_estimator=None,
                      score_precision=5):
        def row(key, scores, params):
            d = {
                 'estimator': key,
                 'mean_score': round(np.mean(scores), score_precision),
                 'std_score': round(np.std(scores), score_precision),
                 'min_score': round(min(scores), score_precision),
                 'max_score': round(max(scores), score_precision),
            }
            return pd.Series({**params, **d})

        rows = []
        for k in self.grid_searches:
            params = self.grid_searches[k].cv_results_['params']
            scores = []
            for i in range(self.grid_searches[k].cv):
                key = "split{}_test_score".format(i)
                r = self.grid_searches[k].cv_results_[key]
                scores.append(r.reshape(len(params), 1))

            all_scores = np.hstack(scores)
            for p, s in zip(params, all_scores):
                rows.append((row(k, s, p)))

        df = pd.concat(rows, axis=1).T.sort_values([sort_by], ascending=False)

        columns = ['estimator', 'mean_score', 'std_score',
                   'min_score', 'max_score']
        columns = columns + [c for c in df.columns if c not in columns]

        if num_rows_per_estimator:
            df = df.groupby('estimator')
            df = df.head(num_rows_per_estimator)

        return df[columns]

In [0]:
models = {
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'LinearRegression': LinearRegression()
}

alphas = np.linspace(0.1, 10, 100)

params = {
    'Lasso': {
        'alpha': alphas,
        'normalize': [False, True]
    },
    'Ridge': {
        'alpha': alphas,
        'normalize': [False, True]
    },
    'LinearRegression': {
        'normalize': [False, True]
    }
}

grid = EstimatorSelectionHelper(models, params)
grid.fit(X_train, Y_train, n_jobs=-1)

gs = grid.score_summary(sort_by='mean_score',
                        num_rows_per_estimator=5)

Running GridSearchCV for Ridge
Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 240 tasks      | elapsed:    3.8s


Running GridSearchCV for Lasso
Fitting 3 folds for each of 200 candidates, totalling 600 fits


[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    5.8s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 330 tasks      | elapsed:    3.9s


Running GridSearchCV for LinearRegression
Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Done 600 out of 600 | elapsed:    5.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:    1.9s finished
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: FutureWarning:

Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.




In [0]:
# ploting power !
fig = go.Figure(
    data=[
        go.Table(
            header=dict(
                values=list(
                    gs.columns
                )
            ),
            cells=dict(
                values=[
                    gs.estimator,
                    gs.mean_score,
                    gs.std_score,
                    gs.min_score,
                    gs.max_score,
                    gs.alpha,
                    gs.normalize
                ]
            )
        )
    ]
)
fig.show()

In [0]:
# initialize classifiers
classifiers = {
    f'ridge_alpha={0.1}': Ridge(alpha=0.1, normalize=True),
    f'lasso_alpha={1}': Lasso(alpha=1, normalize=True),
    'linear_reg': LinearRegression()
}

results = []

# train classifiers and predict both train/test data sets
for clf_name, clf in classifiers.items():
    clf.fit(X_train, Y_train)
    results.append({
        'name': clf_name,
        'train_score': clf.score(X_train, Y_train),
        'test_score':  clf.score(X_test, Y_test)
    })

results = pd.DataFrame(results)

# ploting power !
fig = go.Figure(
    data=[
        # train scores
        go.Bar(
            x=results['name'],
            y=results['train_score'],
            name='Train score',
            text=round(results['train_score'], 5),
            textposition='auto'
        ),
        # test scores
        go.Bar(
            x=results['name'],
            y=results['test_score'],
            name='Test score',
            text=round(results['test_score'], 5),
            textposition='auto'
        ),
    ],
    layout={
        'legend': {
            'orientation': 'h'
        }
    }
)
fig.show()

# **Conclusions de la partie 2**

* il est particulièrement difficile de tirer des conclusions des expérimentations ci-dessus

* on ne voit pas particulièrement de différences entre les 3 modèles, si ce n'est que la paramètre de régularisation $ \alpha $ du Lasso est toujours optimal autour de 1 - ce qui en soit n'est pas une généralité mais une spécifité du dataset étudié

* il peut être intéressant de modifier l'étude pour comparer l'écart-type des prédictions des modèles. Normalement, pour un paramètre $ \alpha $ optimal, les régularisations doivent offrir de meilleures performances en généralisation que la régression linéaire non régularisée. Ce qui peut ne pas être le cas pour tous les datasets.

In [0]:
# TODOS
# ---
# 1) normaliser les données
# from sklearn.preprocessing import StandardScaler, scale
# 2) modifier la méthode de gridsearch pour jouer sur la séparation train/test
# https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/
# 3) aller plus en profondeur dans l'étude des régularisations